In [6]:
!curl -L -O http://217.154.76.42:8080/static/vault.wasm --output-dir attatchments/vault/
!curl -L -O http://217.154.76.42:8080/static/vault.js --output-dir attatchments/vault/
!curl -L -O http://217.154.76.42:8080/static/main.js --output-dir attatchments/vault/

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2293  100  2293    0     0   9616      0 --:--:-- --:--:-- --:--:--  9594
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  119k  100  119k    0     0   195k      0 --:--:-- --:--:-- --:--:--  195k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1564  100  1564    0     0   5989      0 --:--:-- --:--:-- --:--:--  5992


In [3]:
!file attatchments/vault/vault.wasm

attatchments/vault/vault.wasm: WebAssembly (wasm) binary module version 0x1 (MVP)


In [6]:
!wasm-decompile attatchments/vault/vault.wasm -o attatchments/vault/vault.dcmp 
!grep -A 20 -B 1 "check" attatchments/vault/vault.dcmp 


export function check_pin(a:int):int {
  var b:int = g_a;
  var c:int = 32;
  var d:int = b - c;
  g_a = d;
  d[6]:int = a;
  var e:int = d[6]:int;
  var f:int = f_c(e);
  var g:int = 1;
  var h:int = f & g;
  if (eqz(h)) goto B_b;
  var i:int = 0;
  var j:int = 1;
  var k:int = i & j;
  d[31]:byte = k;
  goto B_a;
  label B_b:
  var l:int = d[6]:int;
  f_d(l);
  var m:int = d[6]:int;
  f_e(m);


# Let's just bruteforce the right pin :)

In [7]:
!wasm-interp attatchments/vault/vault.wasm -r check_pin -a "i32:0"

check_pin(i32:0) => i32:0


In [8]:
10**8

100000000

Learning:
    
Using `{0..100000000}` will crash bash as it fully expands the sequence => Just do a normal count

In [1]:
%%bash
for (( i=10; i >= 1; --i ));
do
    ( wasm-interp /home/kali/Developer/ctf/imaginaryctf/attatchments/vault/vault.wasm -r check_pin -a "i32:${i}" | grep "=> i32:1") &
done
wait

## It is painfully slow

1. I suspect that each process creation takes way to much time
2. No runtime optimizations because we only interpret it once 

What do we learn? Just do it in the browser because it 

1. has the right environment
2. natively support WASM
3. can do runtime optimizations

```javascript
function checkPin() {
    const pin = document.getElementById('pin').value.padStart(8, '0');
    if (pin.length !== 8 || !/^\d+$/.test(pin)) {
        showResult("Invalid PIN format", false);
        return;
    }

    const result = Module.ccall('check_pin', 'boolean', ['number'], [parseInt(pin)]);

    fetch('/verify', {
        method: 'POST',
        headers: {'Content-Type': 'application/x-www-form-urlencoded'},
        body: `pin=${pin}&result=${result}`
    })
    .then(response => response.json())
    .then(data => {
        showResult(data.success ? data.flag : data.message, data.success);
    });
}
```

The we can call the WASM function via `Module.ccall('check_pin', 'boolean', ['number'], [parseInt(pin)]);` 

So going into the dev console ... 

```javascript
function test() {
    let start = performance.now();
    for (i = 1; i < 1e8; i++) {

        if (Module.ccall('check_pin', 'boolean', ['number'], [i])) {
            console.log(i)
        }
    }
    return performance.now() - start;
}
test();
```

which returns for us

```
52057891
9029 
```

IT ONLY TOOK 9 seconds. And the is the pin `52057891` which we exchange for the flag